# Scraping with Python

## Why scrape the web?
- Vast source of information
- Automate tasks
- Keep up with sites
- Fun!

*** Can you think of examples ? ***

## Read and Tweet!

![ReadTweet](https://static01.nyt.com/packages/images/developer/readtweet2.jpg
 "We read we tweet")

* by Justin Blinder
* https://justin.work/#/we-read-we-tweet/

## Twitter Sentiments

![TwitterSentiments](http://www.csc.ncsu.edu/faculty/healey/tweet_viz/figs/tweet-viz-ex.png
 "Twitter Sentiments")

* by Healey and Ramaswamy
* http://www.csc.ncsu.edu/faculty/healey/tweet_viz/tweet_app/

## Python data scraping

Copyrights and permission:
* Be careful and polite
* Give credit
* Care about media law
* Don't be evil (no spam, overloading sites, etc.)

## Robots.txt

![Robots.txt](http://blogtimenow.com/wp-content/uploads/2013/11/custom-robots.txt-file-blogger.jpg "Robots.txt")

* Specified by web site owner
* Gives instructions to web robots (aka your script)
* Is located at the top-level directory of the web server

Example:

http://google.com/robots.txt

Things to consider:

* Can be just ignored
* Can be a security risk 


## Scraping with Python:

Scraping is all about HTML tags

There are two basic tasks when we scrape web sites:
1. load a web page to a string
2. parse HTML from a web page to locate the interesting bits

### Bad news: 
* Need to learn about tags
* Websites can be ugly

### HTML

* HyperText Markup Language
* Standard for creating webpages
* HTML tags 
    - have angle brackets
    - typically come in pairs

Here is an example for a minimal webpage defined in HTML tags. The root tag is `<html>` and then you have the `<head>` tag. This part of the page typically includes the title of the page and might also have other meta information like the author or keywords that are important for search engines. The `<body>` tag marks the actual content of the page. You can play around with the `<h2>` tag trying different header levels. They range from 1 to 6. 

In [ ]:
from IPython.display import HTML

s = """<!DOCTYPE html>
<html>
  <head>
    <title>This is a title</title>
  </head>
  <body>
    <h2> Test </h2>
    <p>Hello world!</p>
  </body>
</html>"""

h = HTML(s)
h

### Useful Tags

* heading
`<h1></h1> ... <h6></h6>`

* paragraph
`<p></p>` 

* line break
`<br>` 

* link with attribute

`<a href="http://www.example.com/">An example link</a>`


### Example of a beautifully simple webpage:

http://www.crummy.com/software/BeautifulSoup

### Good news: 

* Some browsers help
* Look for: inspect element
* Need only basic HTML
    
*** Try 'Ctrl-Shift I' in Chrome ***

*** Try 'Command-Option I' in Safari ***



## Useful libraries for scraping with Python

* urllib
* requests
* python3-bs4 (Beautiful Soup)
* lxml
* ...
    

The following cell reads the data from a url using the `urllib` library. You see that we got the whole HTML content of the page into the string variable `source`.

In [ ]:
import urllib

url = 'http://www.crummy.com/software/BeautifulSoup'
#source = urllib.urlopen(url).read()
response = urllib.request.urlopen(url)
bytes = response.read()
response.close()
type(bytes)

Note that 'bytes' isn't a string type yet. We can change that.

In [ ]:
source = bytes.decode("utf8")
type(source)

In [ ]:
print(source)

### Quiz :
* Is the word 'Alice' mentioned on the beautiful soup homepage?
* How often does the word 'Soup' occur on the site?
    - hint: use `.count()`

In [ ]:
## is 'Alice' in source?
print('Alice' in source)

## count occurences of 'Soup'
print(source.count('Soup'))

## Beautiful Soup

* Designed to make your life easier
* Many good functions for parsing HTML code


Recall the two basic tasks when we scrape web sites:
1. load a web page to a string - just did that with `urllib.urlopen(url)`
2. parse HTML from a web page to locate the interesting bits

Let's create a Beautiful Soup object from the string variable `source`.

In [ ]:
import bs4 #this is Beautiful Soup

soup = bs4.BeautifulSoup(source, "lxml")

Compare these two print statements. Note that the `prettify()` function formats the output to show the different levels of the HTML code. 

In [ ]:
## first print statement
print(soup)

In [ ]:
## second print statement
print(soup.prettify())

The 'a' tags are interesting. They indicate links.

Here's how to find all of the 'a' tags:

In [ ]:
soup.findAll('a')

In [ ]:
soup.findAll('Soup')

The last command only returns an empty list, because `Soup` is not an HTML tag. It's just a string that occurs in the webpage.

We can do some other intersting things with our Beautiful Soup element `soup`.

Get the first occurrence of something.

In [ ]:
first_tag = soup.find('a')
first_tag

First `href`:

In [ ]:
first_tag.get('href')

Get all links in the page.

In [ ]:
link_list = [l.get('href') for l in soup.findAll('a')]
link_list

Filter all external links

Don't be alarmed - this is going to throw an error.

In [ ]:
# create an empty list to collect the valid links
external_links = []

# write a loop to filter the links
# if it starts with 'http' we are happy

#Note that this will throw an error!!

for l in link_list:
    if l[:4] == 'http':
        external_links.append(l)

This throws an error! Let's investigate.

Have a closer look at `link_list`:

In [ ]:
link_list

Aha! There are some 'None' elements. Let's verify:

In [ ]:
sum([l is None for l in link_list])

There are two elements in the list that are None! Let's filter those out in the `for` loop:

In [ ]:
external_links = []

for l in link_list:
    if l != None and l[:4] == 'http':
        external_links.append(l)
        
external_links

Note: The above `if` condition works because of lazy evaluation in Python. The `and` statement becomes `False` if the first part is `False`, so there is no need to ever evaluate the second part. Thus a `None` entry in the list gets never asked about its first four characters. 

Here's a more elegant version:

In [ ]:
external_links = [l for l in link_list if l != None and l.startswith('http')]
external_links

### Recap

With just a few lines of Python code, we have extracted all of the external links from a webpage. We could do more extensive parsing of the HTML 'tree,' but let's move on for now.

## Getting Data with an API
* API: application programming interface
* Some sites try to make your life easier
* Twitter, New York Times, IMDb, Yelp, ...

### Before we go further, we need to review Python dictionaries and introduce the JSON data format.

## Python Dictonaries

* Built-in data type
* Uses key: value pairs

In [ ]:
a = {'a': 1, 'b':2}
a

Show keys

In [ ]:
a.keys()

Show values

In [ ]:
a.values()

Loop over all entries - two ways:

First, use `zip` - also works for iterating over any other two lists

In [ ]:
for k,v in zip(a.keys(), a.values()):
    print(k,v)

Next, use `items()`

In [ ]:
for k,v in a.items():
    print(k,v)

## JSON

* JavaScript Object Notation
* Human readable
* Transmit attribute-value pairs

In [ ]:
a = {'a': 1, 'b':2}

In [ ]:
type(a)

In [ ]:
import json

s = json.dumps(a)

In [ ]:
type(s)

s is a string containing a in JSON encoding

In [ ]:
s

In [ ]:
a2 = json.loads(s)

In [ ]:
type(a2)

Reading back, the keys are now in unicode.

In [ ]:
a2

## Twitter

* API a bit more complicated
* Libraries make life easier
* twython:
https://twython.readthedocs.io/en/latest/usage/starting_out.html

Keys: https://developer.twitter.com

### twython!

Python is one of the programming languages with the biggest number of developed wrappers for the Twitter API.
Two popular Python library wrappers for the Twitter APIs are ‘twython’ and ‘tweepy’.
We’ll be using twython. It’s an actively maintained, pure Python wrapper for the Twitter API. It supports both normal (search) and streaming Twitter APIs.

`conda install twython`

## Let's explore Twitter using Python/twython

### Connect to Twitter

To connect to Twitter, we will create an authenticated connection. This will make use of the authentication keys and tokens you established when you signed up as a Twitter developer. If you have forgotten to write them down or save them in a file, you can find them at https://developer.twitter.com/en/apps. Just select your App, and look at the Keys and tokens tab.

### Caveat!

Just to be clear, the Python code for assigning variables in the cell below uses a very bad practice when it comes to sensitive data (i.e. secret credentials). But I’m assuming you just want to get to the action – luckily, it’s easy to revoke/reset/recreate the credentials if they become compromised.

In [ ]:
##Note that the single quote marks are required to be sure these are text strings
CONSUMER_KEY = 'your-app-consumer-key'
CONSUMER_SECRET = 'your-app-consumer-secret'
OAUTH_ACCESS_TOKEN = 'your-access-token'
OAUTH_ACCESS_TOKEN_SECRET = 'your-access-token-secret'

##Instantiate an API client object via the Twython library
from twython import Twython
client = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_ACCESS_TOKEN, OAUTH_ACCESS_TOKEN_SECRET)

Let's see what type of object we've created.

In [ ]:
type(client)

This instantiation of an API client won’t give any indication that client has successfully authorized.

So in Twython, an easy check is to call the `get_account_settings method`. If you get an error message, it means you need to doublecheck those credential strings you assigned to the constants/arguments above.

In [ ]:
client.get_account_settings()

Here is a real test. You can post a status update!

In [ ]:
mytweet = client.update_status(status='Hello World from Python/twython!')

The `update_status` method returns a dictionary, which we’ve assigned in the above snippet to the `mytweet` variable. Every status (i.e. tweet) has an id value:

In [ ]:
mytweet['id']

The corresponding URL for any tweet, given its id, takes this format:

https://twitter.com/YOUR_USERNAME/statuses/TWEET_ID

Check it out with your own `mytweet` id.

### About that earlier caveat ...

Rather than expose your sensitive credentials in a Jupyter notebook or Python script, it is better to store them in a file. One suitable way is to store them in a JSON file "twitter_credentials.json" and load these values from your file when needed.

Here is some code to create the file. Note that I'm using the key and token variables created earlier. You would replace these with your own.

In [ ]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY'] = CONSUMER_KEY  
credentials['CONSUMER_SECRET'] = CONSUMER_SECRET  
credentials['ACCESS_TOKEN'] = OAUTH_ACCESS_TOKEN  
credentials['ACCESS_SECRET'] = OAUTH_ACCESS_TOKEN_SECRET

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)


When you need these credentials, this code works:

In [ ]:
from twython import Twython  
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
client = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'], creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])

In [ ]:
type(client)

### Note
You can learn more about a Twython client object in the official documentation:
https://twython.readthedocs.io/en/latest/.

We can do a lot of things with our client object.

### Do a simple search

We'll just search the main Twitter feed.

In [ ]:
results = client.search(q='#GoBeavs', count=10)
all_tweets = results['statuses']

for tweet in all_tweets:
    print(tweet['text'])

### Let's expand on this a bit. 

The Twython `search` method which returns a dictionary of `search_metadata` and `statuses` - the queried results. We'll only look at the `statuses` part, and save a portion of all information in a `pandas` dataframe, to present it in a table.

Don't be concerned in some of this code looks new to you.

In [ ]:
# Create a query
query = {'q': 'learn python',  
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }

In [ ]:
# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}  
for status in client.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

In [ ]:
import pandas as pd

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  
df.head(5)  

That looks much cleaner, and we can use all of our `pandas` tools.

### Try some more things

In [ ]:
# We'll start over
from twython import Twython
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_ACCESS_TOKEN, OAUTH_ACCESS_TOKEN_SECRET)

Verify credentials.

In [ ]:
twitter.verify_credentials()

Get your "home" timeline:

In [ ]:
twitter.get_home_timeline()

Get the user timeline with screen_name = 'OregonStateMBA'

In [ ]:
twitter_statuses = twitter.get_user_timeline(screen_name='OregonStateMBA', count=100)
len(twitter_statuses)

In [ ]:
type(twitter_statuses)

The first tweet in the 'OregonStateMBA' timeline:

In [ ]:
twitter_statuses[0]

The screen name of the user who wrote the first tweet in that timeline:

In [ ]:
twitter_statuses[0]['user']['screen_name']

We can create a pandas DataFrame timeline tweets. First, we create a Series. Then we use it to create the DataFrame.

In [ ]:
import pandas as pd
pdSeriesList = [pd.Series(t) for t in twitter_statuses]

In [ ]:
df = pd.DataFrame(pdSeriesList)
df.head(2)

Suppose we just want to consider things that have been interesting enough to be retweeted at least 10 times.

In [ ]:
may_be_interesting = df[df.retweet_count>10]
len(may_be_interesting)

Hmmm ... that's not very many!

Let's try all of this for 'BillGates':

In [ ]:
twitter_statuses = twitter.get_user_timeline(screen_name='BillGates', count=100)
pdSeriesList = [pd.Series(t) for t in twitter_statuses]
df = pd.DataFrame(pdSeriesList)
df.head(2)

In [ ]:
may_be_interesting = df[df.retweet_count>10]
len(may_be_interesting)

Wow! That guy gets retweeted!

In [ ]:
df.retweet_count.hist()

In [ ]:
df.retweet_count.max()

OK, let's narrow it down a bit:

In [ ]:
interesting = df[df.retweet_count>14000]
len(interesting)

Get the text of these tweets:

In [ ]:
tweet_text = interesting.text

In [ ]:
tweet_text

In [ ]:
for t in tweet_text.values:
    print('######')
    print(t)

### Searching hashtags

Search for the latest tweets about #GRAMMYs

In [ ]:
q = 'GRAMMYs'
search_results = twitter.search(q=q)
statuses = search_results['statuses']

Here is a very handy function for extracting tweet entities:

In [ ]:
def extract_tweet_entities(statuses):
    
    # See https://dev.twitter.com/docs/tweet-entities for more details on tweet
    # entities

    if len(statuses) == 0:
        return [], [], [], []
    
    screen_names = [ user_mention['screen_name'] 
                         for status in statuses
                            for user_mention in status['entities']['user_mentions'] ]
    
    hashtags = [ hashtag['text'] 
                     for status in statuses 
                        for hashtag in status['entities']['hashtags'] ]

    urls = [ url['expanded_url'] 
                     for status in statuses 
                        for url in status['entities']['urls'] ]
    
    symbols = [ symbol['text']
                   for status in statuses
                       for symbol in status['entities']['symbols'] ]

    return screen_names, hashtags, urls, symbols


Now let's use it for the tweets returned by our hashtag search.

In [ ]:
screen_names, hashtags, urls, symbols = extract_tweet_entities(statuses)

Here are the first five items for each ...

In [ ]:
print("Screen names:")
print(json.dumps(screen_names[0:5], indent=1))
print("Hashtags:")
print(json.dumps(hashtags[0:5], indent=1))
print("URLs:")
print(json.dumps(urls[0:5], indent=1))
print("Symbols:")
print(json.dumps(symbols[0:5], indent=1))

### See what's trending

One way to see what's trending is to use Where On Earth IDs.

See: https://dev.twitter.com/docs/api/1.1/get/trends/place and http://developer.yahoo.com/geo/geoplanet/ for details.

In [ ]:
WORLD_WOE_ID = 1
world_trends = twitter.get_place_trends(id=WORLD_WOE_ID)
print(json.dumps(world_trends, indent=1))

In [ ]:
US_WOE_ID = 23424977
us_trends = twitter.get_place_trends(id=WORLD_WOE_ID)
print(json.dumps(world_trends, indent=1))

### Twitter live stream

It is also possible to connect to a live Twitter stream. Unfortunately, this doesn't work well in a Jupyter Notebook. But here is some code that you can use. 

Again, don't worry if you don't understand all of the code.

In [ ]:
from twython import TwythonStreamer

# Now set up how you want to handle the signals.
class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        if 'text' in data:
            print(data['text'])

    def on_error(self, status_code, data):
        print(status_code)

        # Want to stop trying to get data because of the error?
        # Uncomment the next line!
        # self.disconnect()
        
# Filter public statuses
stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_ACCESS_TOKEN, OAUTH_ACCESS_TOKEN_SECRET)
stream.statuses.filter(track='twitter')

With this code, statuses should be flowing in.

## Conclusion

That's all for now. This brief overview has shown you that Python provides a rich set of tools for scraping data from the Web.